In [44]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
import itertools
from itertools import product
import torch

print('mps: ', torch.backends.mps.is_available())

options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use the Metal backend
else:
    device = torch.device("cpu")  # Fallback to CPU

mps:  True


In [45]:
class BundledChoice_KP:
    def __init__(self,  num_agents , num_objects , lambda_star , random_seed=4,  sigma = 1, max_capacity = 100):

        self.K_MOD = len(lambda_star)
        self.num_agents = num_agents
        self.num_objects = num_objects
        self.random_seed = random_seed

        # Initialize stuff
   

        # True parameters
        self.lambda_star_np = lambda_star
        self.lambda_star = torch.tensor(lambda_star, device=device, dtype=torch.float)

        # Set manual seed
        torch.manual_seed(self.random_seed)

        ### Modular characteristics
        self.φ_i_j_k = torch.normal(0, 1, size=(self.num_agents, self.num_objects, self.K_MOD), device=device, dtype=torch.float)


        ### Knapsack Constraints
        self.weight_j = torch.randint(1, 100, (self.num_objects,), device=device, dtype=torch.float)
        self.capacity_i = torch.randint(1, max_capacity, (self.num_agents,), device=device, dtype=torch.float)
        
    

        ### estimation
        self.φ_hat_k = None
        self.φ_hat_i_k = None
        self.value_LP = None

        self.eps_i_j = sigma * torch.normal(0, 1, size=(self.num_agents, self.num_objects), device=device, dtype=torch.float)

In [ ]:
def linear_knapsack(self, idx, lambda_k , max_weight = None):
    
    if max_weight is None:
        max_weight = int(self.capacity_i[idx].cpu().numpy()[0])

    values_i_j = torch.einsum('i,jk->ijk', self.φ_i_j_k[idx], lambda_k)
    


In [79]:
example_pb = BundledChoice_KP( 5, 10, [1, 2, 3, 4, 5], max_capacity= 200)
idx = [1, 2]
lambda_k = example_pb.lambda_star
print(example_pb.capacity_i[idx].cpu().numpy())
max_weight = int(example_pb.capacity_i[idx].cpu().numpy()[0])
print('max_weight: ', max_weight)
values_i_j = example_pb.φ_i_j_k[idx]  @  lambda_k

weight_states = torch.arange(max_weight + 1, device= device).unsqueeze(0)

[161. 105.]
max_weight:  161


In [74]:
values_i_j

tensor([[  5.0901,  16.9645,  13.3604, -12.2743,   5.1205,  -9.1279,   5.7840,
          19.2348,   4.4172,   6.8700],
        [ -4.4021,   4.2688,   3.8711,  -7.8447,  -6.3756,  -2.0179,  -7.3205,
          -6.7593,   0.6237,  -1.7122]], device='mps:0')

In [ ]:
dp_i_j_w = torch.zeros((len(idx), example_pb.num_objects, max_weight) ,device=device, dtype=torch.float)

In [ ]:
for j in example_pb.num_objects:
    
    mask = weight_states >= example_pb.weight_j[j]
    